## Preprocessing script to ingest raw data from the physician labels

In [1]:
import pandas as pd
import numpy as np

File paths to be changed

In [2]:
# data path and sheet name for the physician labels
# replace with the excel file name
DATA_FILE_PATH = '../data/physician_labels/Labels_TrainImages_1-200 - B - LPL done20250209.xlsx'
# replace with the excel sheet name we want to translate the labels from
DATA_SHEET_NAME = 'Labels_TrainImages_1'

# file path for the translation file for features and labels
TRANSLATION_FILE = '../data/translation/Tongue Features - 4.xlsx'
# sheet name with the translations
TRANSLATION_SHEET_NAME = 'Translated'

# file names of images to remove
IMAGES_TO_REMOVE = []

# export preprocessed file path
EXPORT_PREPROCESSED_PATH = '../data/processed/physician_1.csv'

Read in data

In [3]:
# perform translation of labels
# read in the features and their translations we are concerned with (the column names)
features = pd.read_excel(
    io=TRANSLATION_FILE,
    sheet_name=TRANSLATION_SHEET_NAME,
    engine='openpyxl',
    usecols='A:B',
    nrows=25
)

# read in the labels and their translations we are concerned with (the values for all features)
labels = pd.read_excel(
    io=TRANSLATION_FILE,
    sheet_name=TRANSLATION_SHEET_NAME,
    engine='openpyxl',
    usecols='D:E',
    nrows=24
)

# read in the data
df = pd.read_excel(
    io=DATA_FILE_PATH,
    sheet_name=DATA_SHEET_NAME,
    engine='openpyxl',
    usecols='A:Z',
    nrows=200
)

Preliminary check on whether the features, labels, and data are of correct size

In [ ]:
def check_features_and_labels(features, labels):
    '''
    Assertions to check integrity of data for the features and labels. This is to ensure proper parsing of features and labels.
    '''
    # check shape and no null values
    assert features.isnull().sum().sum() == 0, "Features dataframe contains null values"
    assert features.shape == (25, 2), "Features dataframe has incorrect shape"

    assert labels.isnull().sum().sum() == 0, "Labels dataframe contains null values"
    assert labels.shape == (24, 2), "Labels dataframe has incorrect shape"

def check_data(df):
    '''
    Assertions to check integrity of physician labelled data. This is to ensure proper parsing of data.
    '''
    # assuming that we always have 200 images in each batch and 25 features for the model
    assert df.isnull().sum().sum() == 0, "Physician labelled data contains null values"
    assert df.shape == (200, 26), "Physician labelled data has incorrect shape"

In [5]:
check_features_and_labels(features, labels)
check_data(df)

In [6]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Column_Chinese  25 non-null     object
 1   Column_English  25 non-null     object
dtypes: object(2)
memory usage: 532.0+ bytes


In [7]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Values_Chinese  24 non-null     object
 1   Values_English  24 non-null     object
dtypes: object(2)
memory usage: 516.0+ bytes


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  200 non-null    object
 1   舌色          200 non-null    object
 2   舌质          200 non-null    object
 3   瘀斑-中        200 non-null    object
 4   瘀斑-边        200 non-null    object
 5   点刺/舌红-尖     200 non-null    object
 6   点刺/舌红-边     200 non-null    object
 7   点刺-中间散在     200 non-null    object
 8   裂纹-中        200 non-null    object
 9   裂纹-根        200 non-null    object
 10  齿痕-尖        200 non-null    object
 11  齿痕-边        200 non-null    object
 12  舌苔厚度-左上     200 non-null    object
 13  舌苔厚度-右上     200 non-null    object
 14  舌苔厚度-右下     200 non-null    object
 15  舌苔厚度-左下     200 non-null    object
 16  舌苔湿度        200 non-null    object
 17  舌苔密度-左上     200 non-null    object
 18  舌苔密度-右上     200 non-null    object
 19  舌苔密度-右下     200 non-null    object
 20  舌苔密度-左下   

In [9]:
df.shape # correct shape, since 200 images and 26 columns

(200, 26)

Correct number of labels and features

## Perform the translation of data in the dataframe

In [10]:
feature_dict = features.set_index('Column_Chinese')['Column_English'].to_dict()
# add a dummy key for image_name so that it does not map to nan
feature_dict['image_name'] = 'image_name'

label_dict = labels.set_index('Values_Chinese')['Values_English'].to_dict()

In [11]:
df.columns = df.columns.map(feature_dict)
df.head()

,image_name,Tongue Colour,Tongue Texture,Petechiae - centre,Petechiae - sides,Speckled or red - tip,Speckled or red - sides,Speckled - centre,Tongue Cracks - centre,Tongue Cracks - back,...,Coating Moisture Level,Coating Density - top left,Coating Density - top right,Coating Density - bottom right,Coating Density - bottom left,Scanting Coating,Coating Colour - top left,Coating Colour - top right,Coating Colour - bottom right,Coating Colour - bottom left
0,CC_1.jpg,红,中等,无,无,有,有,无,无,无,...,润,腻,腻,无腐腻,无腐腻,无,白,白,白,白
1,CC_100.jpg,淡红,中等,无,无,有,无,无,有,无,...,燥,无腐腻,无腐腻,无腐腻,无腐腻,无,黄,黄,白,白
2,CC_1001.jpg,红,中等,无,无,有,有,无,有,有,...,润,腻,腻,无腐腻,无腐腻,无,白,白,白,白
3,CC_1002.jpg,淡红,中等,无,无,无,无,无,有,有,...,润,腻,腻,无腐腻,无腐腻,无,黄,黄,白,白
4,CC_101.jpg,淡红,中等,无,无,无,无,无,有,无,...,润,无腐腻,无腐腻,无腐腻,无腐腻,无,白,白,白,白


In [12]:
# loop to loop through all columns and apply translation
for c in df.columns:
    # if it is image name, do not apply any translation
    if c != 'image_name':
        # translate
        df[c] = df[c].map(label_dict)

df.head()

,image_name,Tongue Colour,Tongue Texture,Petechiae - centre,Petechiae - sides,Speckled or red - tip,Speckled or red - sides,Speckled - centre,Tongue Cracks - centre,Tongue Cracks - back,...,Coating Moisture Level,Coating Density - top left,Coating Density - top right,Coating Density - bottom right,Coating Density - bottom left,Scanting Coating,Coating Colour - top left,Coating Colour - top right,Coating Colour - bottom right,Coating Colour - bottom left
0,CC_1.jpg,red,normal,absent,absent,present,present,absent,absent,absent,...,Moist,Greasy,Greasy,non-greasy,non-greasy,absent,White,White,White,White
1,CC_100.jpg,pale red,normal,absent,absent,present,absent,absent,present,absent,...,Dry,non-greasy,non-greasy,non-greasy,non-greasy,absent,Yellow,Yellow,White,White
2,CC_1001.jpg,red,normal,absent,absent,present,present,absent,present,present,...,Moist,Greasy,Greasy,non-greasy,non-greasy,absent,White,White,White,White
3,CC_1002.jpg,pale red,normal,absent,absent,absent,absent,absent,present,present,...,Moist,Greasy,Greasy,non-greasy,non-greasy,absent,Yellow,Yellow,White,White
4,CC_101.jpg,pale red,normal,absent,absent,absent,absent,absent,present,absent,...,Moist,non-greasy,non-greasy,non-greasy,non-greasy,absent,White,White,White,White


In [13]:
# dataframe does not have any null values, so we have performed translation properly
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   image_name                        200 non-null    object
 1   Tongue Colour                     200 non-null    object
 2   Tongue Texture                    200 non-null    object
 3   Petechiae - centre                200 non-null    object
 4   Petechiae - sides                 200 non-null    object
 5   Speckled or red - tip             200 non-null    object
 6   Speckled or red - sides           200 non-null    object
 7   Speckled - centre                 200 non-null    object
 8   Tongue Cracks - centre            200 non-null    object
 9   Tongue Cracks - back              200 non-null    object
 10  Tooth-marks - tip                 200 non-null    object
 11  Tooth-marks - sides               200 non-null    object
 12  Coating Thickness - to

## Ignore the other images we do not want, drop them

In [ ]:
# TODO: need to discuss with Yan Chun the rules on what to remove

## Remove hashtags and special formatting of image files

In [15]:
df['image_name'] = df['image_name'].str.replace(r'\d*#', '', regex=True)

## Export the processed csv file

In [16]:
# check if data is ok before exporting
check_data(df)

In [17]:
df.to_csv(EXPORT_PREPROCESSED_PATH, index=False)